In [9]:
import sys,os

import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

#this works apparently only for savefig stuff
mpl.rcParams['figure.figsize']=(15,10)    #(6.0,4.0)
mpl.rcParams['font.size']=10                #10 
mpl.rcParams['savefig.dpi']=400             #72 
mpl.rcParams['figure.subplot.bottom']=.1    #.125


plt.rc('font', family='serif')
plt.rc('text', usetex=False)

#inline Shit
#%matplotlib inline
%config InlineBackend.figure_format='svg'
%config InlineBackend.rc = {'figure.facecolor': 'white', 'figure.subplot.bottom': 0.125, 'figure.edgecolor': 'white', 'savefig.dpi': 400, 'figure.figsize': (12.0, 8.0), 'font.size': 10}

#GUi shit
#%matplotlib tk

mpl.get_configdir()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys,os

import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import openpiv.tools
import openpiv.process
import openpiv.scaling
from skimage import img_as_int
    
deltaT=1e-3; # Framerate of the sequence

frame_a  = img_as_int(openpiv.tools.imread('experiment1_0781.jpg')).astype(np.int32) # male picture from float64 to int16 (try to use int32 FIX!)
frame_b  = img_as_int(openpiv.tools.imread('experiment1_0782.jpg')).astype(np.int32)

/usr/local/lib/python3.4/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '
/usr/local/lib/python3.4/site-packages/skimage/util/dtype.py:107: UserWarning: Possible precision loss when converting from float64 to int16
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [6]:
plt.matshow( frame_a, cmap=cm.Greys_r );
plt.matshow( frame_b, cmap=cm.Greys_r );
plt.show()

In [7]:
window_size= 64;
spacing = 8;
overlap = window_size-spacing;


u, v, sig2noise = openpiv.process.extended_search_area_piv(frame_a, 
                                                           frame_b, 
                                                           window_size=window_size, 
                                                           overlap=overlap, 
                                                           dt=deltaT, 
                                                           search_area_size=64, 
                                                           sig2noise_method='peak2peak')

x, y = openpiv.process.get_coordinates( image_size=frame_a.shape, window_size=window_size, overlap=overlap )
print(sig2noise)

u2 = u.copy();
v2 = v.copy();
u2, v2, mask = openpiv.validation.sig2noise_val( u2, v2, sig2noise, threshold = 0.5)





[[ 1.82731753  1.95708485  2.06395948 ...,  3.21266267  3.35435095  0.        ]
 [ 1.83495295  1.98662032  2.13006007 ...,  3.18386036  3.24186093  0.        ]
 [ 1.95295651  2.0767292   2.22603798 ...,  3.25042215  3.16943413  0.        ]
 ..., 
 [ 2.30726743  2.41086969  2.35412656 ...,  3.35761704  3.28136079  0.        ]
 [ 2.39104789  2.42921505  2.43881485 ...,  3.09558212  3.18191636  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [8]:
u3 = u2.copy();
v3 = v2.copy();
u3, v3 = openpiv.filters.replace_outliers( u3, v3, method='localmean', max_iter=10, kernel_size=2 )

In [9]:
plt.close("all")

fig = plt.figure(figsize=(15,10));


ax0 = fig.add_subplot(131)
ax0.set_autoscale_on(False)
ax0.set_title("PIV Extended Search Area" )


ax0.matshow( frame_a, cmap=cm.Greys_r );
ax0.quiver( x, y, u, v , color='r')

ax1 = fig.add_subplot(132, sharex=ax0, sharey=ax0)
ax1.set_autoscale_on(False)
ax1.set_title("Outlier Rejected" )

ax1.matshow( frame_a, cmap=cm.Greys_r );
ax1.quiver( x, y, u2, v2 , color='r')

ax3 = fig.add_subplot(133, sharex=ax0, sharey=ax0)
ax3.set_autoscale_on(False)
ax3.set_title("Replaced Outlier" )

ax3.matshow( frame_a, cmap=cm.Greys_r );
ax3.quiver( x, y, u3, v3 , color='r')


ax0.set_xlim([0,frame_a.shape[1]])
ax0.set_ylim([0,frame_a.shape[0]])
ax1.set_xlim([0,frame_a.shape[1]])
ax1.set_ylim([0,frame_a.shape[0]])
fig.tight_layout()
plt.show()

/usr/local/lib/python3.4/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")


In [10]:
window_size= 64;
spacing = 8;
overlap = window_size-spacing;

u, v, sig2noise = openpiv.pyprocess.piv(frame_a, 
                                        frame_b, 
                                        window_size=window_size, 
                                        overlap=overlap, 
                                        dt=deltaT, 
                                        corr_method='fft',
                                        sig2noise_method='peak2peak')

x, y = openpiv.process.get_coordinates( image_size=frame_a.shape, window_size=window_size, overlap=overlap )

u2 = u.copy();
v2 = v.copy();
u2, v2, mask = openpiv.validation.sig2noise_val( u2, v2, sig2noise, threshold = 1.0);

In [14]:

fig = plt.figure(figsize=(15,10));


ax0 = fig.add_subplot(121)
ax0.set_autoscale_on(False)

ax0.matshow( frame_a, cmap=cm.Greys_r );
ax0.quiver( x, y, u, v , color='r')
ax0.set_title("PIV Extended Search Area" )

ax1 = fig.add_subplot(122, sharex=ax0, sharey=ax0)
ax1.set_autoscale_on(False)
ax1.set_title("Outlier Rejected" )
ax1.matshow( frame_a, cmap=cm.Greys_r );
ax1.quiver( x, y, u2, v2 , color='r')


ax0.set_xlim([0,frame_a.shape[1]])
ax0.set_ylim([0,frame_a.shape[0]])
ax1.set_xlim([0,frame_a.shape[1]])
ax1.set_ylim([0,frame_a.shape[0]])
fig.tight_layout()
plt.show()

/usr/local/lib/python3.4/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
